In [7]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

df = pd.read_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Director-Firm-Years\0109director-firm-years_delta.csv", sep=";", decimal='.', dtype={"DIR_Nationality": str}) #DtypeWarning: Columns (195) have mixed types.Specify dtype option on import 
df

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_active,PF_max_gains_per_director,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,PF_gain_loss_replacement_indicator,PF_gainer,PF_looser,PF_changer,FIRM_presence_in_PF,years_before_gain,years_after_loss,FIRM_gain_prior_vs_event_dummy,FIRM_loss_prior_vs_event_dummy,solo_observation_dummy,PF_no_of_dirs_selfcalculated,PF_no_of_dirs_selfcalculated_delta,PF_number_of_dirs,PF_number_of_dirs_delta,PF_no_of_dirs_selfcalc_ego_delta,PF_no_of_dirs_selfcalc_ego,gain_happened_max_changes,gain_happened_dummy,loss_happened_min_changes,loss_happened_dummy,PF_Ego_Market_Rep_Average,PF_MarketCap_Total_ego,PF_MarketCap_withoutEvent_ego,PF_Ego_Market_Rep_Avg_delta_prev,PF_Ego_Market_Rep_Avg_delta_prev_rel,PF_Ego_Market_Rep_Avg_delta_prev_rel_dummy,delta_MarketCap_ego,delta_MarketCap_ego_dummy,PF_MarketCapTotal,PF_MarketCap_Average_woEventFirm,delta_FIRM_PF_mcap_woFirm_REVERSE,delta_FIRM_PF_mcap_w0_woFirm_REVERSE,delta_FIRM_PF_mcap_woFirm_dummy_REVERSE,delta_FIRM_PF_mcap__w0_woFirm_dummy_REVERSE,PF_MarketCapAverage,FIRM_market_cap_total,delta_FIRM_to_avg_PF_mcap_DELETE,delta_FIRM_to_avg_PF_mcap_dummy_DELETE,delta_FIRM_to_avg_PF_mcap_manual0,delta_FIRM_PF_mcap_manual0_REVERSE,delta_FIRM_PF_mcap_manual0_dummy,delta_FIRM_PF_mcap_manual0_dummy_REVERSE,delta_FIRM_PF_beta,delta_FIRM_PF_beta_REVERSE,delta_FIRM_PF_beta_dummy,delta_FIRM_PF_beta_dummy_REVERSE,delta_FIRM_PF_bmeetings,delta_FIRM_PF_bmeetings_REVERSE,delta_FIRM_PF_beetings_dummy,delta_FIRM_PF_bmeetings_dummy_REVERSE,delta_FIRM_PF_comp,delta_FIRM_PF_comp_REVERSE,delta_FIRM_PF_comp_dummy,delta_FIRM_PF_comp_dummy_REVERSE,delta_FIRM_PF_GD_rating_overall,PF_MarketCapAverage_delta_to_previous_value,PF_MarketCapAverage_delta_to_previous_value_relative,PF_MarketCapAverage_delta_to_previous_value_dummy_DELETE_includes_firstObs,PF_MarketCapAverage_delta_to_previous_value_dummy,PF_market_cap_delta_mlogit_dummy,PF_avg_previouspf_valued_at_currentyear,PF_avg_currentpf,PF_delta_previousvaluedatcurrent_minus_current,PF_delta_previousvaluedatcurrent_minus_current_realtivfe,PF_avg_currentpf_delta_to_previous_value,PF_event_gain_dummy_circularity,PF_event_loss_dummy_circularity,PF_event_replace_dummy_circularity,PF_event_change_dummy_circularity,PF_avg_current_valued_in_prioryear,PF_beta_average,PF_beta_total,PF_vola_average,PF_vola_total,PF_tobinsQ_average,PF_ROA_average,PF_EPS_average,PF_board_meetings_average,PF_attendance_level,PF_independence_level_average,PF_accruals_change_average,PF_total_assets_average,PF_growth_assets_average,PF_impairment_intangibles_average,PF_cash_average,PF_impairment_goodwill_average,PF_debt_to_capital_average,PF_closely_held_shares_average,PF_amort_intang_goodwill_average,PF_amort_of_def_charges_averages,PF_amort_intang_average,PF_common_shares_out_average,PF_bribery_corruption_fraud_dummy,PF_research_exp_average,PF_early_auditor_resign_dummy,PF_managment_departure_dummy,PF_goodwill_writtenoff_average,PF_earnings_restatement_average,PF_governance_score_average,PF_board_size_average,PF_NED_board_size_average,PF_board_compensation_average,PF_is_chairman_dummy,PF_GD_rating_overall_average_withEventFirm,PF_GD_rating_balance_average_withEventFirm,PF_GD_rating_culture_average_withEventFirm,PF_GD_rating_career_average_withEventFirm,PF_GD_rating_comp_average_withEventFirm,PF_GD_rating_mgmt_average_withEventFirm,PF_GD_recommends_average_withEventFirm,PF_GD_positive_outlook_average_withEventFirm,PF_GD_approves_of_CEO_average_withEventFirm,PF_GD_helpful_average_withEventFirm,PF_GD_current_employee_average_withEventFirm,PF_GD_full_time_average_withEventFirm,PF_GD_tenure_average_withEventFirm,PF_GD_rating_overall_count_average_withEventFirm,PF_GD_rating_overall_sum_withEventFirm,PF_GD_rating_balance_sum_withEventFirm,PF_GD_rating_culture_sum_withEventFirm,PF_GD_rating_career_sum_withEventFirm,PF_GD_rating_comp_sum_withEventFirm,PF_GD_rating_mgmt_sum_withEventFirm,PF_GD_recommends_sum_withEventFirm,PF_GD_positiv

In [8]:
print([i for i in df.columns if "board" in i])

['PF_board_meetings_average', 'PF_board_size_average', 'PF_NED_board_size_average', 'PF_board_compensation_average', 'FIRM_board_meetings', 'FIRM_board_independence', 'FIRM_board_attendance', 'FIRM_board_size', 'FIRM_board_compensation']


### groupby multiple variables and multiple functions

In [10]:
df_grouped = df.groupby(["DirectorID", "Year"], as_index=False).agg({'FIRM_beta': [lambda x: x.sum(min_count=1), "count"], 'FIRM_board_meetings': [lambda x: x.sum(min_count=1), "count"], 'FIRM_board_compensation': [lambda x: x.sum(min_count=1), "count"], "FIRM_ESG_Score": [lambda x: x.sum(min_count=1), "count"]}) #lambda x: x.sum(min_count=1) -> "sum" but only when obs >= 1 for that group

grouped_col_names = [f"{x}_{y}" if "FIRM_" in x else x for x, y in df_grouped.columns.to_flat_index()]
grouped_col_names = [sub.replace('<lambda_0>', 'sum') for sub in grouped_col_names] #custom agg function adds weird substring
grouped_col_names = [sub.replace('FIRM_', 'PF_') for sub in grouped_col_names] #replace FIRM_ with PF_ for aggregated vars
df_grouped.columns = grouped_col_names # 2 headers (multi columns) -> one header. x -> var name. y -> func name (eg sum)

df_relevant = df[["DirectorID", "ISIN", "Year", "FIRM_presence_in_PF", "PF_no_of_dirs_selfcalc_ego", 'FIRM_beta', 'FIRM_board_meetings', 'FIRM_board_compensation', "FIRM_ESG_Score"]]

df_merge = df_relevant.merge(df_grouped, how="left", on=["DirectorID", "Year"])
df_merge

# df_merge[df_merge["FIRM_beta_sum"].isna()]

,DirectorID,ISIN,Year,FIRM_presence_in_PF,PF_no_of_dirs_selfcalc_ego,FIRM_beta,FIRM_board_meetings,FIRM_board_compensation,FIRM_ESG_Score,PF_beta_sum,PF_beta_count,PF_board_meetings_sum,PF_board_meetings_count,PF_board_compensation_sum,PF_board_compensation_count,PF_ESG_Score_sum,PF_ESG_Score_count
0,216931,US6369183024,1999,NaN,NaN,3.67,NaN,NaN,NaN,3.67,1,NaN,0,NaN,0,NaN,0
1,216931,US6369183024,2000,NaN,NaN,-1.48,NaN,NaN,NaN,-1.48,1,NaN,0,NaN,0,NaN,0
2,216931,US6369183024,2001,NaN,NaN,-0.94,NaN,NaN,NaN,-0.94,1,NaN,0,NaN,0,NaN,0
3,216931,US6369183024,2002,NaN,NaN,-0.15,NaN,NaN,NaN,-0.15,1,NaN,0,NaN,0,NaN,0
4,216931,US6369183024,2003,NaN,NaN,0.12,NaN,NaN,NaN,0.12,1,NaN,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916288,206877812921,KYG2345N1025,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0
916289,206877812921,KYG2345N1025,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0
916290,206877812921,KYG2345N1025,2017,2.0,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0
916291,206877812921,KYG2345N1025,2018,1.0,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0


### calculate delta variables for multiple vars

In [14]:
firm_values = ["FIRM_beta", "FIRM_board_meetings", "FIRM_board_compensation", "FIRM_ESG_Score"]
pf_values_withoutEvent = ["PF_beta_withoutEvent", "PF_board_meetings_withoutEvent", "PF_board_compensation_withoutEvent", "PF_ESG_Score_withoutEvent"]
pf_values_sum = ["PF_beta_sum", "PF_board_meetings_sum", "PF_board_compensation_sum", "PF_ESG_Score_sum"]
pf_values_count = ["PF_beta_count", "PF_board_meetings_count", "PF_board_compensation_count", "PF_ESG_Score_count"]
delta_values = ["delta_beta", "delta_board_meetings", "delta_board_compensation", "delta_ESG_Score"]
delta_dummy = ["delta_beta_dummy", "delta_board_meetings_dummy", "delta_board_compensation_dummy", "delta_ESG_Score_dummy"]

for i in range(0,4):
    #calculate: (pf avg - firm rating) / (no_dirs - 1) if dir-year obs has more than 2 firms in his gd pf
    df_merge[pf_values_withoutEvent[i]] = np.where(df_merge["PF_no_of_dirs_selfcalc_ego"] >= 2, ( df_merge[pf_values_sum[i]] - df_merge[firm_values[i]] ) / ( df_merge["PF_no_of_dirs_selfcalc_ego"] - 1 ), np.nan )
    #if event firm is not present in PF during a specific year, then just calc average based on existing PF / number of dirs (without adjusting by minus 1)
    df_merge[pf_values_withoutEvent[i]] = np.where(df_merge["FIRM_presence_in_PF"].isna() == True, ( df_merge[pf_values_sum[i]] ) / ( 1 ), df_merge[pf_values_withoutEvent[i]] )

    #calc delta: firm minus pf rating
    df_merge[delta_values[i]] = np.where(df_merge[pf_values_withoutEvent[i]].isna() == False, df_merge[firm_values[i]] - df_merge[pf_values_withoutEvent[i]], np.nan)

    #calc dummy: 1 if delta > 0 ; 0 if delta < 0
    # df_merge.insert(df_merge.columns.get_loc("delta_MarketCap") + 1, "delta_MarketCap_dummy", np.nan)
    df_merge[delta_dummy[i]] = np.where((df_merge[delta_values[i]].isna() == False) & (df_merge[delta_values[i]] > 0), 1, #not nan and larger 0
        np.where((df_merge[delta_values[i]].isna() == False) & (df_merge[delta_values[i]] < 0), 0, np.nan)) #not nan and smaller 0. 0 is set to nan (cases in which pf value equals firm value, which dont make sense)

df_merge

In [18]:
df_final = df.merge(df_merge, how='left', on=["DirectorID", "ISIN", "Year"], suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)') #drop duplicate columns of the merge operation

#throw around columns based on lists of prior cell UNFINISHED
# for i in range(0,4):
#     #cut column
#     col = df_final.pop(firm_values[i])
#     #insert column
#     df_final.insert(df_final.columns.get_loc("PF_MarketCap_Total") + i + 1, col.name, col) 

df_final

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_active,PF_max_gains_per_director,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,PF_gain_loss_replacement_indicator,PF_gainer,PF_looser,PF_changer,FIRM_presence_in_PF,years_before_gain,years_after_loss,FIRM_gain_prior_vs_event_dummy,FIRM_loss_prior_vs_event_dummy,solo_observation_dummy,PF_no_of_dirs_selfcalculated,PF_no_of_dirs_selfcalculated_delta,PF_number_of_dirs,PF_number_of_dirs_delta,PF_no_of_dirs_selfcalc_ego_delta,PF_no_of_dirs_selfcalc_ego,gain_happened_max_changes,gain_happened_dummy,loss_happened_min_changes,loss_happened_dummy,PF_Ego_Market_Rep_Average,PF_MarketCap_Total_ego,PF_MarketCap_withoutEvent_ego,PF_Ego_Market_Rep_Avg_delta_prev,PF_Ego_Market_Rep_Avg_delta_prev_rel,PF_Ego_Market_Rep_Avg_delta_prev_rel_dummy,delta_MarketCap_ego,delta_MarketCap_ego_dummy,PF_MarketCapTotal,PF_MarketCap_Average_woEventFirm,delta_FIRM_PF_mcap_woFirm_REVERSE,delta_FIRM_PF_mcap_w0_woFirm_REVERSE,delta_FIRM_PF_mcap_woFirm_dummy_REVERSE,delta_FIRM_PF_mcap__w0_woFirm_dummy_REVERSE,PF_MarketCapAverage,FIRM_market_cap_total,delta_FIRM_to_avg_PF_mcap_DELETE,delta_FIRM_to_avg_PF_mcap_dummy_DELETE,delta_FIRM_to_avg_PF_mcap_manual0,delta_FIRM_PF_mcap_manual0_REVERSE,delta_FIRM_PF_mcap_manual0_dummy,delta_FIRM_PF_mcap_manual0_dummy_REVERSE,delta_FIRM_PF_beta,delta_FIRM_PF_beta_REVERSE,delta_FIRM_PF_beta_dummy,delta_FIRM_PF_beta_dummy_REVERSE,delta_FIRM_PF_bmeetings,delta_FIRM_PF_bmeetings_REVERSE,delta_FIRM_PF_beetings_dummy,delta_FIRM_PF_bmeetings_dummy_REVERSE,delta_FIRM_PF_comp,delta_FIRM_PF_comp_REVERSE,delta_FIRM_PF_comp_dummy,delta_FIRM_PF_comp_dummy_REVERSE,delta_FIRM_PF_GD_rating_overall,PF_MarketCapAverage_delta_to_previous_value,PF_MarketCapAverage_delta_to_previous_value_relative,PF_MarketCapAverage_delta_to_previous_value_dummy_DELETE_includes_firstObs,PF_MarketCapAverage_delta_to_previous_value_dummy,PF_market_cap_delta_mlogit_dummy,PF_avg_previouspf_valued_at_currentyear,PF_avg_currentpf,PF_delta_previousvaluedatcurrent_minus_current,PF_delta_previousvaluedatcurrent_minus_current_realtivfe,PF_avg_currentpf_delta_to_previous_value,PF_event_gain_dummy_circularity,PF_event_loss_dummy_circularity,PF_event_replace_dummy_circularity,PF_event_change_dummy_circularity,PF_avg_current_valued_in_prioryear,PF_beta_average,PF_beta_total,PF_vola_average,PF_vola_total,PF_tobinsQ_average,PF_ROA_average,PF_EPS_average,PF_board_meetings_average,PF_attendance_level,PF_independence_level_average,PF_accruals_change_average,PF_total_assets_average,PF_growth_assets_average,PF_impairment_intangibles_average,PF_cash_average,PF_impairment_goodwill_average,PF_debt_to_capital_average,PF_closely_held_shares_average,PF_amort_intang_goodwill_average,PF_amort_of_def_charges_averages,PF_amort_intang_average,PF_common_shares_out_average,PF_bribery_corruption_fraud_dummy,PF_research_exp_average,PF_early_auditor_resign_dummy,PF_managment_departure_dummy,PF_goodwill_writtenoff_average,PF_earnings_restatement_average,PF_governance_score_average,PF_board_size_average,PF_NED_board_size_average,PF_board_compensation_average,PF_is_chairman_dummy,PF_GD_rating_overall_average_withEventFirm,PF_GD_rating_balance_average_withEventFirm,PF_GD_rating_culture_average_withEventFirm,PF_GD_rating_career_average_withEventFirm,PF_GD_rating_comp_average_withEventFirm,PF_GD_rating_mgmt_average_withEventFirm,PF_GD_recommends_average_withEventFirm,PF_GD_positive_outlook_average_withEventFirm,PF_GD_approves_of_CEO_average_withEventFirm,PF_GD_helpful_average_withEventFirm,PF_GD_current_employee_average_withEventFirm,PF_GD_full_time_average_withEventFirm,PF_GD_tenure_average_withEventFirm,PF_GD_rating_overall_count_average_withEventFirm,PF_GD_rating_overall_sum_withEventFirm,PF_GD_rating_balance_sum_withEventFirm,PF_GD_rating_culture_sum_withEventFirm,PF_GD_rating_career_sum_withEventFirm,PF_GD_rating_comp_sum_withEventFirm,PF_GD_rating_mgmt_sum_withEventFirm,PF_GD_recommends_sum_withEventFirm,PF_GD_positiv

In [19]:
df_final.to_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Director-Firm-Years\0109director-firm-years_gamma.csv", sep=";", decimal='.', index=False)